In [1]:
import json
from tqdm import tqdm

In [2]:
import torch
from torchvision import transforms
import timm
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from PIL import Image
import os
import random

/Users/viktor/anaconda3/envs/cv_vkusvill/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
random.randint(1,100)

77

In [4]:
import wandb

# Log in to your W&B account
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: uimanov-viktor (mdma). Use `wandb login --relogin` to force relogin


True

In [5]:
wandb.init(project='CV fruits', entity='mdma')

In [6]:
num_epochs = 100
wandb.config = {
  "learning_rate": 0.001,
  "epochs": num_epochs,
  "batch_size": 32
}

In [7]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import timm
import os

# Step 1: Define a Custom Dataset Class
class CustomDataset(Dataset):
    def __init__(self, image_dir, annotations, transform=None):
        self.image_dir = image_dir
        self.annotations = annotations
        self.transform = transform

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        img_name = self.annotations[idx]['file_name']
        img_path = os.path.join(self.image_dir, img_name)
        image = Image.open(img_path).convert('RGB')

        # Save the original image for visualization
        original_image = transforms.ToTensor()(image)
        original_image = transforms.Resize((224,224))(original_image)

        if self.transform:
            image = self.transform(image)
        
        return image, original_image ,self.annotations[idx]['category_id']

# Step 2: Define Transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # EfficientNet-B0 expects 224x224 images
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


In [8]:
model = timm.create_model('efficientnet_b0', pretrained=True)
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"Using device: {device}")
num_classes = 5  # Update with the number of your classes
model.classifier = nn.Linear(model.classifier.in_features, num_classes)
model.to(device)

Using device: mps


EfficientNet(
  (conv_stem): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn1): BatchNormAct2d(
    32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
    (drop): Identity()
    (act): SiLU(inplace=True)
  )
  (blocks): Sequential(
    (0): Sequential(
      (0): DepthwiseSeparableConv(
        (conv_dw): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (bn1): BatchNormAct2d(
          32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): SiLU(inplace=True)
        )
        (se): SqueezeExcite(
          (conv_reduce): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
          (act1): SiLU(inplace=True)
          (conv_expand): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
          (gate): Sigmoid()
        )
        (conv_pw): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn2): BatchNormAct2d(
      

In [9]:

with open('/Users/viktor/data_science/CV/vkusvill/Apple-Disease-Detection/train/_annotations.coco.json', 'r') as file:
    coco_data = json.load(file)
image_id_to_file_name = {img['id']: img['file_name'] for img in coco_data['images']}

# Create the annotations list with file names and category ids
annotations = [{'file_name': image_id_to_file_name[ann['image_id']], 'category_id': ann['category_id']} for ann in coco_data['annotations']]
image_dir = '/Users/viktor/data_science/CV/vkusvill/Apple-Disease-Detection/train_cropped'
dataset = CustomDataset(image_dir=image_dir, annotations=annotations, transform=transform)
data_loader = DataLoader(dataset, batch_size=32, shuffle=True)

# Step 4: Load a Pretrained EfficientNet Model


# Step 1: Load the validation annotations
with open('/Users/viktor/data_science/CV/vkusvill/Apple-Disease-Detection/valid/_annotations.coco.json') as f:
    valid_annotations = json.load(f)

image_id_to_file_name_valid = {img['id']: img['file_name'] for img in valid_annotations['images']}

# Create the annotations list with file names and category ids
annotations_valid = [{'file_name': image_id_to_file_name_valid[ann['image_id']], 'category_id': ann['category_id']} for ann in valid_annotations['annotations']]

valid_dataset = CustomDataset(
    image_dir='/Users/viktor/data_science/CV/vkusvill/Apple-Disease-Detection/valid', 
    annotations=annotations_valid, 
    transform=transform
)

# Step 2: Define a validation data loader
valid_data_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False)


In [10]:
categories_names = {cat['id']:cat['name'] for cat in coco_data['categories']}

In [12]:


# Step 5: Define Loss Function and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
# Watch the model
wandb.watch(model, log='all')

# Prepare the validation dataset and dataloader
# Assuming valid_dataset is already created using your CustomDataset class
valid_data_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False)

# Training loop
for epoch in range(num_epochs):
    model.train()
    running_train_loss = 0.0
    for batch_idx, (inputs,_,labels) in enumerate(data_loader):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        train_loss = criterion(outputs, labels)
        train_loss.backward()
        optimizer.step()
        running_train_loss += train_loss.item()
    
    # Average training loss for the epoch
    avg_train_loss = running_train_loss / len(data_loader)
    
    # Validation loop
    model.eval()
    with torch.no_grad():
        for inputs, original_images, labels in valid_data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            val_loss = criterion(outputs, labels)
            
            # Get the predictions
            _, preds = torch.max(outputs, 1)
            
            # Log the average validation loss for the batch
            avg_val_loss = val_loss.item()

            # Choose one image to log
            idx_to_log = random.randint(0, len(original_images) - 1) # Index of the image you want to log
            # Assume the original_images tensor is already on CPU and normalized
            logged_image = original_images[idx_to_log].to(device)
            logged_image_pil = transforms.ToPILImage()(logged_image)

            # Log validation loss and the selected image to wandb
            wandb.log({
                "train_loss":avg_train_loss,
                "val_loss": avg_val_loss,
                "val_image": wandb.Image(logged_image, caption=f"True: {categories_names[labels[idx_to_log].item()]}, Pred: {categories_names[preds[idx_to_log].item()]}")
            })

            # Break after logging one image and the loss
            break

# Finish the run
wandb.finish()



wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


train_loss,█▃▁▂▂▁▁▂▂▁▂▁▁▁▂▁▁▁▁▁▂▁▁▂▁▁▁▁▂▁▁▁▁▂▃▂▁▁▁▁
val_loss,▂▄▅▁▁▂▃▆▁▁▁▄▃▄█▃▃▂▃▃▃▇▄▃▂▂▁▂▃▂▂▃▅▇▆▃▂▅▄▅
train_loss,0.02814
val_loss,1.19931


In [18]:
preds

tensor([[-8., -1., -6.,  3.,  1.],
        [-8., -1., -6.,  3.,  1.],
        [-8., -1., -6.,  3.,  1.],
        [-8., -1., -6.,  3.,  1.],
        [-8., -1., -6.,  3.,  1.],
        [-7., -7., -8.,  8.,  0.],
        [-7.,  0., -2.,  2., -1.],
        [-7.,  0., -2.,  2., -1.]], device='mps:0')

In [19]:
wandb.finish()

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


train_loss,█▃▃▂▂▁▁▂▂▂▂▂▁▁▁▁
val_loss,▄▁▂▁▂▁▂▄▂▂█▂▂▂▃▅
train_loss,0.01683
val_loss,0.99151


In [14]:
torch.save(model.state_dict(), 'model_weights.pth')


In [19]:
# Containers for predictions and true labels
all_preds = torch.tensor([], dtype=torch.int64)
all_true_labels = torch.tensor([], dtype=torch.int64)

# Iterate over the evaluation data
with torch.no_grad():
    for inputs, _, labels in valid_data_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        # Generate predictions
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)  # Use torch.max for multi-class classification
        
        # Store predictions and true labels
        all_preds = torch.cat((all_preds, preds.cpu()), 0)
        all_true_labels = torch.cat((all_true_labels, labels.cpu()), 0)

# Calculate True Positives and False Positives
true_positives = torch.sum((all_preds == 1) & (all_true_labels == 1)).item()
false_positives = torch.sum((all_preds == 1) & (all_true_labels == 0)).item()

# Calculate precision
precision = true_positives / (true_positives + false_positives + 1e-8)  # Add a small epsilon to avoid division by zero

print(f'Precision: {precision:.4f}')


/Users/viktor/anaconda3/envs/cv_vkusvill/lib/python3.11/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


Precision: 1.0000


In [20]:
all_preds

tensor([3, 1, 3, 4, 2, 3, 1, 1, 1, 1, 4, 4, 4, 1, 4, 1, 3, 4, 2, 4, 3, 2, 3, 3,
        1, 4, 4, 3, 3, 3, 3, 3, 2, 2, 2, 3, 3, 4, 3, 4, 3, 3, 3, 1, 3, 4, 4, 3,
        2, 2, 3, 4, 3, 3, 4, 1, 3, 3, 2, 3, 3, 4, 3, 2, 3, 3, 3, 3, 3, 3, 3, 3])

In [21]:
all_true_labels

tensor([3, 1, 1, 4, 2, 1, 1, 1, 1, 1, 4, 4, 4, 1, 4, 1, 3, 4, 2, 4, 3, 2, 1, 4,
        3, 1, 4, 3, 1, 3, 1, 3, 2, 2, 2, 3, 1, 4, 3, 4, 2, 2, 2, 1, 3, 4, 4, 4,
        2, 2, 1, 4, 3, 3, 4, 4, 3, 3, 2, 3, 3, 4, 4, 2, 2, 2, 2, 2, 2, 1, 2, 2])

In [26]:
categories_names

{0: 'Apple-Disease', 1: 'BLOTCH', 2: 'HEALTHY', 3: 'ROT', 4: 'SCAB'}

In [34]:
# Class of interest
class_of_interest = 3

# True Positives (TP): The model predicted the class of interest and it was correct
true_positives = torch.sum((all_preds == class_of_interest) & (all_true_labels == class_of_interest))

# False Positives (FP): The model predicted the class of interest but it was incorrect
false_positives = torch.sum((all_preds == class_of_interest) & (all_true_labels != class_of_interest))

# Precision for the class of interest
precision = true_positives.float() / (true_positives + false_positives).float()

print(f'Precision for class {class_of_interest}: {precision:.4f}')

Precision for class 3: 0.4167


In [41]:
def calculate_precision_recall_f1(y_true, y_pred, class_num):
    """
    Calculate precision, recall, and F1 score for a specific class.
    
    Parameters:
    y_true (torch.Tensor): Ground truth labels.
    y_pred (torch.Tensor): Predictions made by the model.
    class_num (int): The class number for which metrics are calculated.
    
    Returns:
    precision (float): The precision for the specified class.
    recall (float): The recall for the specified class.
    f1_score (float): The F1 score for the specified class.
    """
    # True Positives (TP): The model predicted the class correctly
    true_positives = torch.sum((y_pred == class_num) & (y_true == class_num))
    
    # False Positives (FP): The model predicted the class, but it was incorrect
    false_positives = torch.sum((y_pred == class_num) & (y_true != class_num))
    
    # False Negatives (FN): The model did not predict the class, but it was the true class
    false_negatives = torch.sum((y_pred != class_num) & (y_true == class_num))
    
    # Calculate precision and recall
    precision = true_positives.float() / (true_positives + false_positives).float() if true_positives + false_positives > 0 else 0.0
    recall = true_positives.float() / (true_positives + false_negatives).float() if true_positives + false_negatives > 0 else 0.0
    
    # Calculate F1 score
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0
    
    return precision.item(), recall.item(), f1_score.item()





In [40]:
categories_names

{0: 'Apple-Disease', 1: 'BLOTCH', 2: 'HEALTHY', 3: 'ROT', 4: 'SCAB'}

In [45]:

# Example usage:
class_num = 4  # Class of interest
precision, recall, f1_score = calculate_precision_recall_f1(all_true_labels, all_preds, class_num)

print(f'Precision for class {class_num}: {precision:.4f}')
print(f'Recall for class {class_num}: {recall:.4f}')
print(f'F1 Score for class {class_num}: {f1_score:.4f}')

Precision for class 4: 0.9375
Recall for class 4: 0.7895
F1 Score for class 4: 0.8571
